In [5]:
import os
from shutil import copyfile
import glob

# Deep leearnig and machine learning libraries (Keras and scikit learn)
from os import listdir
from tensorflow.keras import datasets, layers, models, utils
from keras.utils import to_categorical
from tensorflow.python.framework import ops
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from sklearn.metrics import confusion_matrix , classification_report
# from sklearn.model_selection import cross_val_score
from sklearn.model_selection import  KFold
# Classic libraries with python
import numpy as np
import matplotlib.pyplot as plt
import cv2
%matplotlib inline
from google.colab import drive
drive.mount('/content/drive') 


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
# Settings:
img_size = 64
num_classes = 10
path ='/content/drive/MyDrive/Dataset'



def read_greyimg(data_file):
    #Normalization step1 (convert image to grey)
    img = cv2.imread(data_file,cv2.IMREAD_GRAYSCALE )
    img = cv2.resize(img, (img_size, img_size))
    return img

def first_normalize():
  X = []
  Y = [] 
  labels = listdir(path) 
  for i, label in enumerate(labels):
          dir = path+'/'+label
          for data_file in listdir(dir):
              img = read_greyimg(dir+'/'+data_file)
              X.append(img)
              Y.append(label)
  X = np.array(X)
  Y = np.array(Y)
  Y = to_categorical(Y, num_classes)

  X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size = 0.8)
  # Normalization step2 (Divide each image by 255)
  X_train=X_train/255
  X_test=X_test/255

  #convert 3D to 2D 
  X_train  = X_train.reshape( -1, img_size*img_size)
  X_test= X_test.reshape( -1, img_size*img_size)
  return X_train,Y_train,X_test,Y_test

In [41]:
 # first Neural Network architecture
 X_train ,Y_train,X_test,Y_test = first_normalize() 

 ann1 = models.Sequential([
           layers.Dense(2048,input_shape=(img_size*img_size,), activation='relu'),
           layers.Dense(1024, activation='relu'),
           layers.Dense(10, activation='softmax'),
 ])

 ann1.compile(optimizer='adam',
               loss='categorical_crossentropy',
               metrics=['accuracy'])


# Cross Validation
X_cross=X_train
Y_cross=Y_train
kfolds = KFold(n_splits=3)
i=0
for train_index, test_index in kfolds.split(X_cross):
    i=i+1
    X_train_cross, X_test_cross, Y_train_cross, Y_test_cross = X_cross[train_index], X_cross[test_index],Y_cross[train_index], Y_cross[test_index]
    ann1.fit(X_train_cross, Y_train_cross,epochs=8)
    acc=ann1.evaluate(X_test_cross,Y_test_cross)
    print(acc[1])
    print("acc in split "+ str(i) + " :"+str(acc[1]))



#Evaluate model 1
ann1.summary()
 Y_pred1 = ann1.predict(X_test_cross)
 Y_pred1 = [np.argmax(element) for element in Y_pred1]
 Y_test1 = [np.argmax(element) for element in Y_test_cross]
 print(classification_report(Y_test1, Y_pred1)) 

Epoch 1/8
38/38 [==============================] - 4s 108ms/step - loss: 5.8170 - accuracy: 0.0933
Epoch 2/8
38/38 [==============================] - 4s 111ms/step - loss: 2.3309 - accuracy: 0.1208
Epoch 3/8
38/38 [==============================] - 3s 88ms/step - loss: 2.2515 - accuracy: 0.1517
Epoch 4/8
38/38 [==============================] - 3s 88ms/step - loss: 2.0678 - accuracy: 0.2533
Epoch 5/8
38/38 [==============================] - 3s 90ms/step - loss: 1.8712 - accuracy: 0.3175
Epoch 6/8
38/38 [==============================] - 3s 88ms/step - loss: 1.6687 - accuracy: 0.3775
Epoch 7/8
38/38 [==============================] - 3s 89ms/step - loss: 1.6144 - accuracy: 0.3992
Epoch 8/8
19/19 [==============================] - 1s 29ms/step - loss: 1.3372 - accuracy: 0.4917
0.49166667461395264
acc in split 1 :0.49166667461395264
Epoch 1/8
38/38 [==============================] - 3s 89ms/step - loss: 1.3646 - accuracy: 0.4983
Epoch 2/8
38/38 [==============================] - 3s 90ms/s

In [33]:
# Second Neural Network architecture
X_train ,Y_train,X_test,Y_test = first_normalize() 

ann2 = models.Sequential([
  layers.Dense(100,input_shape=(img_size*img_size,), activation='relu'),
  layers.Dense(400,input_shape=(img_size*img_size,), activation='sigmoid'),
  layers.Dense(300,input_shape=(img_size*img_size,), activation='sigmoid'),
  layers.Dense(10, activation='softmax'),
])

ann2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Cross Validation
X_cross=X_train
Y_cross=Y_train
kfolds = KFold(n_splits=3)
i=0
for train_index, test_index in kfolds.split(X_cross):
    i=i+1
    X_train_cross, X_test_cross, Y_train_cross, Y_test_cross = X_cross[train_index], X_cross[test_index],Y_cross[train_index], Y_cross[test_index]
    ann2.fit(X_train_cross, Y_train_cross,epochs=8)
    acc=ann2.evaluate(X_test_cross,Y_test_cross)
    print("acc in split "+ str(i) + " :"+str(acc[1]))


#Evaluate model 2
ann2.summary()

Y_pred2 = ann2.predict(X_test_cross)
Y_pred2 = [np.argmax(element) for element in Y_pred2]
Y_test2 = [np.argmax(element) for element in Y_test_cross]
print(classification_report(Y_test2, Y_pred2))  


Epoch 1/8
38/38 [==============================] - 1s 8ms/step - loss: 2.3675 - accuracy: 0.1150
Epoch 2/8
38/38 [==============================] - 0s 8ms/step - loss: 2.3437 - accuracy: 0.1133
Epoch 3/8
38/38 [==============================] - 0s 8ms/step - loss: 2.3258 - accuracy: 0.1275
Epoch 4/8
38/38 [==============================] - 0s 7ms/step - loss: 2.3218 - accuracy: 0.1100
Epoch 5/8
38/38 [==============================] - 0s 7ms/step - loss: 2.2623 - accuracy: 0.1625
Epoch 6/8
38/38 [==============================] - 0s 7ms/step - loss: 2.0330 - accuracy: 0.2583
Epoch 7/8
38/38 [==============================] - 0s 8ms/step - loss: 1.7719 - accuracy: 0.3658
Epoch 8/8
19/19 [==============================] - 0s 3ms/step - loss: 1.6548 - accuracy: 0.3950
acc in split 1 :0.39500001072883606
Epoch 1/8
38/38 [==============================] - 0s 9ms/step - loss: 1.4951 - accuracy: 0.4683
Epoch 2/8
38/38 [==============================] - 0s 7ms/step - loss: 1.4004 - accuracy: 0

In [29]:
#Train using normalized RGB image

def read_rgbimg(data_file):
    img = cv2.imread(data_file)
    img = cv2.resize(img, (img_size, img_size))
    return img
def second_normalize():
  X = []
  Y = []
  labels = listdir(path) 
  for i, label in enumerate(labels):
          dir = path+'/'+label
          for data_file in listdir(dir):
              img = read_rgbimg(dir+'/'+data_file)
              X.append(img)
              Y.append(label)
  X = np.array(X).astype('float32')
  Y = np.array(Y)
  Y = to_categorical(Y, num_classes) 

  average= X.mean(axis=0)
  X -= average
  X=X/255
  X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size = 0.8)
  return X_train,Y_train,X_test,Y_test


In [31]:
#CNN model
X_train ,Y_train,X_test,Y_test = second_normalize() 

cnn1 = models.Sequential([
    layers.Conv2D(filters=64, kernel_size=(4, 4), activation='relu', input_shape=(img_size,img_size,3)),
    layers.MaxPooling2D(pool_size=2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])
cnn1.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


# Cross Validation
X_cross=X_train
Y_cross=Y_train
kfolds = KFold(n_splits=3)
i = 0
for train_index, test_index in kfolds.split(X_cross):
    i = i+1
    X_train_cross, X_test_cross, Y_train_cross, Y_test_cross = X_cross[train_index], X_cross[test_index],Y_cross[train_index], Y_cross[test_index]
    cnn1.fit(X_train_cross, Y_train_cross)
    acc=cnn1.evaluate(X_test_cross,Y_test_cross)
    print("acc in split "+ str(i) + " :"+str(acc[1]))




#Evaluate cnn
cnn1.summary()

Y_pred = cnn1.predict(X_test_cross)
Y_pred = [np.argmax(element) for element in Y_pred]
Y_test = [np.argmax(element) for element in Y_test_cross]
print(classification_report(Y_test, Y_pred))  

19/19 [==============================] - 1s 40ms/step - loss: 0.7654 - accuracy: 0.7600
acc in split 1 :0.7599999904632568
19/19 [==============================] - 1s 41ms/step - loss: 0.4265 - accuracy: 0.8567
acc in split 2 :0.8566666841506958
19/19 [==============================] - 1s 42ms/step - loss: 0.2688 - accuracy: 0.9050
acc in split 3 :0.9049999713897705
Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 61, 61, 64)        3136      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                             
                                                                 
 flatten_5 (Flatten)         (None, 57600)             0         
                                                                 
 dense_28 (Den

In [39]:
#SVM vs NN 
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

X_train ,Y_train,X_test,Y_test = second_normalize() 

Y_train=np.argmax(Y_train,axis=1)
Y_test=np.argmax(Y_test,axis=1)


X_train  = X_train.reshape( -1, img_size*img_size*3)#in case of rgb image 
X_test= X_test.reshape( -1, img_size*img_size*3)  

svm = SVC() 
svm.fit(X_train,Y_train)
score = svm.score(X_test,Y_test)
Y_predict= svm.predict(X_test)
print("acc ="+str(score))
print(classification_report(Y_test, Y_predict))
 


acc =0.8558758314855875
              precision    recall  f1-score   support

           0       0.93      0.98      0.95        52
           1       0.93      0.87      0.90        47
           2       0.84      0.89      0.86        46
           3       1.00      0.82      0.90        50
           4       0.76      0.72      0.74        43
           5       0.92      0.97      0.94        59
           6       0.75      0.93      0.83        41
           7       0.77      0.66      0.71        41
           8       0.73      0.77      0.75        35
           9       0.89      0.86      0.88        37

    accuracy                           0.86       451
   macro avg       0.85      0.85      0.85       451
weighted avg       0.86      0.86      0.86       451

